In [11]:
from random import random
from scipy.stats import chi2
from numpy import log

## Ejercicio 1

In [2]:
def T(k, n, p_i, N):
    """
    Estadístico Chi Cuadrado.

    Args:
        k (int): cantidad de valores que puede tomar la variable aleatoria
        n (int): tamaño muestral
        p_i (list[float]): probabilidades teóricas para cada valor que puede
        tomar la distribución de la hipótesis nula
        N (list[int]): frecuencias observadas

    Returns:
        t (float): estadístico Chi Cuadrad observado a partir de la muestra
    """
    t = 0
    for i in range(k):
        t += (N[i] - n*p_i[i])**2 / (n*p_i[i])
    return t

### a) Plantear el test de hipótesis pertinente y realizar el cálculo en papel del estadístico

- **Hipótesis nula**: los datos obtenidos en la muestra siguen la distribución global, esto es:  
&emsp; La proporción de autos de color:  
&emsp;&emsp; - blanco es 0.22  
&emsp;&emsp; - plateado es 0.20  
&emsp;&emsp; - negro es 0.19  
&emsp;&emsp; - gris es 0.12  
&emsp;&emsp; - rojo 0.09  
&emsp;&emsp; - marrón 0.08  
&emsp;&emsp; - verde 0.07  
&emsp;&emsp; - azul 0.02  
&emsp;&emsp; - otros 0.01  

- **Estadístico a utilizar**: como se tratan de datos discretos, el estadístico a utilizar será el del test Chi-cuadrado de Pearson, dado por la siguiente fórmula:
$$
T = \sum_{i=0}^{k} \frac{(N_i - np_i)^2}{np_i}
$$
&emsp; donde:  
&emsp; - k es la cantidad de valores que pueden tomar los datos (en este caso es 9)  
&emsp; - $N_i$ es la frecuencia observada del dato i, es decir la cantidad de veces que aparece el valor i en los datos obtenidos de la muestra.  
&emsp; - n es el tamaño muestral, es decir la cantidad de datos de la muestra.  
&emsp; - $p_i$ es la probabilidad que una variable con la distribución del hipótesis nula tome el valor i.

- **p-valor**: siguiendo el test  Chi-cuadrado de Pearson, el p-valor estará definido de la siguiente forma:  
&emsp; Si T = t es el valor del estadístico, el p-valor es:
$$
P_{H_0}(T \geq t)
$$

In [3]:
n = 539
k = 9
N = [120, 114, 92, 85, 34, 33, 45, 11, 5]
p = [0.22, 0.2, 0.19, 0.12, 0.09, 0.08, 0.07, 0.02, 0.01]

t = T(k, n, p, N)
print(f'Valor observado del estadístico T: {t}')

Valor observado del estadístico T: 15.964309985059911


### b) Dar el p-valor de la prueba y la conclusión que este provee para un nivel de rechazo $\alpha = 0.05$

#### i) utilizando la aproximación con la distribución $\chi^2$

Recordemos que si la hipótesis nula es cierta y n es lo suficientemente grande, entonces el estadístico T tiene distribución $\chi$-cuadrado con k-1 grados de libertad: $\chi_{k-1}^2$. Y luego, teniendo esto en cuenta, y la forma en que definimos el p-valor, tenemos que ahora:
$$
p-valor = P(T \geq t) = P(\chi_{k-1}^2 \geq t)
$$

In [5]:
p_valor_chi_2 = 1 - chi2.cdf(t, k-1)
print(f'p-valor utilizando una aproximación con la distribución Chi-cuadrado: {p_valor_chi_2}')

p-valor utilizando una aproximación con la distribución Chi-cuadrado: 0.04289380378443686


#### ii) realizando 10000 simulaciones

In [10]:
# Necesito un algoritmo que genere valores de la muestra teórica:
def colores():
    U = random()
    if U < 0.22:
        return 0
    elif U < 0.42:
        return 1
    elif U < 0.61:
        return 2
    elif U < 0.73:
        return 3
    elif U < 0.82:
        return 4
    elif U < 0.9:
        return 5
    elif U < 0.97:
        return 6
    elif U < 0.99:
        return 7
    else:
        return 8

n_sim = 10000
p_valor_sim = 0
for _ in range(n_sim):
    X = [colores() for i in range(n)]
    N_sim = [0] * k
    for x in X:
        N_sim[x] += 1
    t_sim = T(k, n, p, N_sim)
    if t_sim >= t:
        p_valor_sim += 1
print(f'p-valor obtenido realizando {n_sim} simulaciones: {p_valor_sim/n_sim}')

p-valor obtenido realizando 10000 simulaciones: 0.0455


## Ejercicio 2

### a) Explicar en qué consiste el método de refinamiento con un proceso de Poisson homogéneo para generar este proceso. Determina la tasa del proceso homogéneo utilizado.


El método de refinamiento es uno que se utiliza para generar un proceso de Poisson **no homogéneo** a partir de la generación de uno **homogéneo**, por lo cual para entenderlo es necesario también saber cómo se genera un proceso de Poisson homogéneo con tasa de intensidad $\lambda$.

Recordemos que en un proceso de Poisson homogéneo con tasa de intensidad $\lambda$, las variables aleatorias que representan los tiempos de llegada entre eventos sucesivos son exponenciales con parámetro $\lambda$. Por lo tanto, para generar los primeros n eventos de un proceso de Poisson homogéneo con tasa de intensidad $\lambda$, basta con generar n exponenciales con parámetro $\lambda$.

Dicho esto, el método de refinamiento para generar un proceso de Poisson homogéneo se basa en la siguiente proposición:
&emsp; Supongamos N(t) un proceso de Poisson homogéneo con tasa de intensidad $\lambda$. Entonces, el proceso M(t) que cuenta los eventos de N(t) con probabilidad p(t) es un proceso de Poisson no homogéneo con intensida $\lambda(t) = \lambda . p(t)$

Teniendo esta proposición en cuenta, si tomamos $p(t) = \frac{\lambda(t)}{\lambda}$ (con $\lambda \geq \lambda(t)$), entonces tendremos un proceso de Poisson no homogéneo con intensidad $\lambda(t)$.

En este, caso tenemos que $\lambda(t) = (t-3)^2, 0 \leq t \leq 6$. Por lo tanto, una tasa de intensidad $\lambda$ que cumple que $\lambda \geq \lambda(t)$ para $0 \leq t \leq 6$ es $\lambda = 9$.

### b) Determinar al menos 4 intervalos para la mejora del algoritmo dada en a). Implementar  en código un programa que simule el proceso con el método de refinamiento mejorado y devuelva una lista de los tiempos de eventos en el intervalo [0,6].

In [13]:
def exponencial(lamda):
    U = random()
    return -log(U)/lamda

def poisson_adelgazamiento_mejorado(lamdas, intervs, lamda_t, T):
    j = 0       # Recorre subintervalos
    t = exponencial(lamdas[j])
    eventos = []
    while t <= T:
        if t <= intervs[j]:
            U = random()
            if U < lamda_t(t) / lamdas[j]:
                eventos.append(round(t,4))
            t += exponencial(lamdas[j])
        else:
            t = intervs[j] + (t - intervs[j]) * lamdas[j] / lamdas[j+1]
            j += 1
    return eventos, len(eventos)

In [15]:
"""
Intervalos propuestos:
[0,1]: acoto con lambda=9
(1,2]: acoto con lambda=4
(2,3]: acoto con lambda=1
(3,4]: acoto con lambda=1
(4,5]: acoto con lambda=4
(5,6]: acoto con lambda=9
"""
lamda_t = lambda x: (x-3)**2
intervs = [1, 2, 3, 4, 5, 6]
lamdas = [9, 4, 1, 1, 4, 9]
T = 6
eventos, _ = poisson_adelgazamiento_mejorado(lamdas, intervs, lamda_t, T)
print(f'Tiempos de eventos hasta el tiempo 6:\n {eventos}')

Tiempos de eventos hasta el tiempo 6:
 [0.0581, 0.5357, 0.6035, 0.6546, 0.6788, 0.7692, 0.9921, 1.068, 1.1145, 1.2413, 1.3048, 1.6286, 3.9448, 4.6539, 4.9057, 4.9977, 5.0945, 5.1063, 5.1098, 5.1431, 5.1489, 5.1835, 5.3124, 5.4858, 5.7469, 5.9544]
